In [2]:
print("OK")

OK


In [3]:
%pwd

'/workspaces/End-to-End-Medical_chatbot_using_RAGS/research'

In [4]:
import os 
os.chdir("../")

In [5]:
%pwd

'/workspaces/End-to-End-Medical_chatbot_using_RAGS'

In [6]:
%pip install langchain-text-splitters
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/workspaces/End-to-End-Medical_chatbot_using_RAGS/medibot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf_files("/workspaces/End-to-End-Medical_chatbot_using_RAGS/data")

In [9]:
extracted_data[:2]  # Display first two documents

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': '/workspaces/End-to-End-Medical_chatbot_using_RAGS/data/Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': '/workspaces/End-to-End-Medical_chatbot_using_RAGS/data/Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')]

In [10]:
len(extracted_data)

637

In [11]:
from typing import List
from langchain_core.documents import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [12]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [13]:
minimal_docs[:2]  # Display the first two minimal documents

[Document(metadata={'source': '/workspaces/End-to-End-Medical_chatbot_using_RAGS/data/Medical_book.pdf'}, page_content=''),
 Document(metadata={'source': '/workspaces/End-to-End-Medical_chatbot_using_RAGS/data/Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')]

In [14]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [15]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 5859


In [16]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

/tmp/ipykernel_23407/2533971096.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [17]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
vector = embedding.embed_query("Hello world")
vector

[-0.03447727859020233,
 0.03102324716746807,
 0.006734943948686123,
 0.02610895037651062,
 -0.03936202824115753,
 -0.1603025197982788,
 0.06692401319742203,
 -0.006441460456699133,
 -0.047450434416532516,
 0.014758843928575516,
 0.07087527215480804,
 0.05552758276462555,
 0.019193317741155624,
 -0.026251306757330894,
 -0.010109581984579563,
 -0.026940494775772095,
 0.022307422012090683,
 -0.022226620465517044,
 -0.1496926248073578,
 -0.017493054270744324,
 0.0076762172393500805,
 0.0543522872030735,
 0.00325441500172019,
 0.03172592446208,
 -0.08462139964103699,
 -0.02940598502755165,
 0.05159560963511467,
 0.048124004155397415,
 -0.00331481802277267,
 -0.05827918276190758,
 0.041969262063503265,
 0.02221064455807209,
 0.128188818693161,
 -0.022338906303048134,
 -0.011656313203275204,
 0.06292835623025894,
 -0.032876331359148026,
 -0.09122605621814728,
 -0.031175334006547928,
 0.052699558436870575,
 0.04703483358025551,
 -0.08420306444168091,
 -0.030056189745664597,
 -0.020744858309626

In [19]:
print( "Vector length:", len(vector))

Vector length: 384


In [20]:
import os
from dotenv import load_dotenv

load_dotenv("/workspaces/End-to-End-Medical_chatbot_using_RAGS/.env")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Set environment variables
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY  # Use GOOGLE_API_KEY instead

print(f"PINECONE_API_KEY loaded: {PINECONE_API_KEY is not None}")
print(f"GOOGLE_API_KEY loaded: {GEMINI_API_KEY is not None}")

PINECONE_API_KEY loaded: True
GOOGLE_API_KEY loaded: True


In [21]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [22]:

from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [23]:

from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [24]:
pc

In [25]:
from pinecone import ServerlessSpec 

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [26]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [27]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [28]:
dswith = Document(
    page_content="This is a sample document for testing.",
    metadata={"source": "Youtube"}
)

In [29]:
docsearch.add_documents(documents=[dswith])

['a9030fd7-5302-44ba-ad36-9d9f067fdb36']

In [30]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [31]:
retrieved_docs = retriever.invoke("what is acne")
retrieved_docs

[Document(id='b75b84db-37c1-480b-bd7d-2d7b715db07b', metadata={'source': '/workspaces/End-to-End-Medical_chatbot_using_RAGS/data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='ed055fc5-2f98-43f6-858c-c9c380985971', metadata={'source': '/workspaces/End-to-End-Medical_chatbot_using_RAGS/data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='95900843-db8b-4878-8818-706c3aec98ca', metadata={'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26')]

In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI

chatModel = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY,
    convert_system_message_to_human=True
)

In [33]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

In [34]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [35]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [36]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

/workspaces/End-to-End-Medical_chatbot_using_RAGS/medibot/lib/python3.12/site-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Acromegaly is a disorder characterized by the abnormal release of a chemical from the pituitary gland in the brain. This leads to increased growth in bone and soft tissue, along with various other disturbances throughout the body. The text does not provide a specific definition for gigantism.


In [ ]:
response = rag_chain.invoke({"input": "what is alcoholism ?"})
print(response["answer"])

Alcoholism is defined by the maladaptive use of alcohol, which is recurrent and significant. The provided text does not offer further details beyond this introductory statement.
